# Checkpoint 2
### Due: Tuesday, 9 November, 2021 at 11:00am BST

### Total points: 100

### Read This First

Wherever you see raise NotImplementedError(), remove that line and put your code there.

Put the code that produces the output for a given task in the cell indicated. You are welcome to add as many cells as you like for imports, function definitions, variables, etc.

Your notebook must run correctly when executed once from start to finish. Your notebook will be graded based on how it runs, not how it looks when you submit it. To test this, go to the Kernel menu and select Restart & Run All.

Once you are happy with it, clear the output by selecting Restart & Clear Output from the Kernel menu.

Submit through Noteable.

# Task 1 (20 points)

The first task is to find the number of local minima of the function in different number of dimensions ( from 1 to 5 inclusive). Your code should return the list or array of 5 integer values.

The number of local minima for ndim = 1 should be 4. All the minimima should be within -10..10 range and there are no minima separated by less than 0.01

In [ ]:
import numpy as np
import scipy.optimize
import scipy.linalg
import matplotlib.pyplot as plt
import scipy.stats
%matplotlib inline

In [ ]:
D = np.load('data.npz')['D']
dims = [1, 2, 3, 4, 5]
import scipy.special as ssp

def blackbox_func(p):
    """
    This is is the function that you need to optimize
    DO NOT CHANGE THE CODE INSIDE THIS FUNCTION
    """
    ndim = len(p)
    pos = dims.index(ndim)
    curD = D[10000 * pos:10000 * (pos + 1), :ndim]
    return -ssp.logsumexp(np.sum(-0.5 * ((p[None, :] - curD) / .3)**2, axis=1))

In [ ]:
D = np.load('data.npz')['D']
dims = [1, 2, 3, 4, 5]
import scipy.special as ssp

def blackbox_func(p):
    """
    This is is the function that you need to optimize
    DO NOT CHANGE THE CODE INSIDE THIS FUNCTION
    """
    ndim = len(p)
    pos = dims.index(ndim)
    curD = D[10000 * pos:10000 * (pos + 1), :ndim]
    return -ssp.logsumexp(np.sum(-0.5 * ((p[None, :] - curD) / .3)**2, axis=1))

In [ ]:
def find_nminima():
    n_minima = []
    for n in dims:
        #minimima within -10..10 range, seperated by 0.01 (which gives 200 divisions)
        x = np.linspace(-10,10,200)
        #initialize empty lists
        minima = []
        results = []
        #iterating over the 200 starting points
        for i in range(200):
            #get the starting points in dimensions of the problem, n dimensions
            x0 = x[i]*np.ones(n)
            #mimize function for different starting points
            res = scipy.optimize.minimize(blackbox_func, x0, method='Nelder-Mead')
            #generate a results vector full of minima of the function
            results.append(res.x[0])
    
        resultsArray = np.array(results)
        #to get the unique minima that are seperated by 0.01 or more, round results to 2dp and get non repeated minima results
        minima.append(np.unique(resultsArray.round(decimals=2)))
        #the number of minima in the given dimension is given by the length of the unique minima vector
        min_in_dims = len(minima[0])
        #generate a list of the number of minima for each dimension
        n_minima.append(min_in_dims)
    return n_minima

In [ ]:
### This function takes approx 2 minutes to run

In [ ]:
assert (len(find_nminima()) == 5)
assert ((find_nminima()[0]) == 4)


# Task 2 (20 pts)

Generate random numbers from a probability distribution that looks like two triangles. 
The PDF f(x) is the following. Use the inverse CDF sampling method.
* f(x) = 0 when x<0
* f(x)= x/2 when 0<x<=1
* f(x)= 1-x/2 when 1<x<=2
* f(x)= 0 when 2<x<=3
* f(x)= (x-3)/2 when 3<x<4
* f(x)= (5-x)/2 when 4<x<5
* f(x)= 0 when x>=5


---

Calculations of the inverse CDF were done on pen and paper

### integrating PDF f(x) to get CDF:
CDF:
1. F(X) = x^2/4 0<x<=1
2. F(X) = -0.5 + x - x^2/4 1<x<=2
3. F(X) = (x^2 - 6x + 10)/4 3<x<4
4. F(X) = (10x - x^2 -23)/4 4<x<5

### INVERTING CDF TO GET INVERSE CDF:
INV CDF:
- F-1(y) = sqrt(4y) 0<y<=0.25
- F-1(y) = 2 - sqrt(8-16y))/2 0.25<y<=0.5
- F-1(y) = 3 + sqrt(-8+16y)/2 0.5<y<0.75
- F-1(y) = 5 - sqrt(16-16y)/2 0.75<y<1

In [ ]:
def generate_triangles(N):
    """ Return a numpy array with the length N with 
    random numbers following the distribution specified
    """
    # YOUR CODE HERE
    #generate a random uniform distribution of numbers
    ys = np.random.uniform(size=N)
    xs = 0*ys
    
    #equations for the inverse cdf for the given y limits as described above
    xs[ys<=0.25] = np.sqrt(4 * ys[ys <= 0.25]) #x at 0 to 1
    xs[(ys>0.25) & (ys<=0.5)] = 2 - np.sqrt(8-16*ys[(ys>0.25) & (ys<=0.5)])/2 #x at 1 to 2
    xs[(ys>0.5) & (ys<0.75)] = 3 + np.sqrt(-8+16*ys[(ys>0.5) & (ys<0.75)])/2 #x at 3 to 4
    xs[ys>0.75] = 5 - np.sqrt(16-16*ys[ys>0.75])/2 #x at 4 to 5
    
    return xs


In [ ]:
# The code will be tested by this
plt.hist(generate_triangles(100000), bins=200);

# Task 3 (20 pts)

The motion of bodies in the Solar system can be described by the Kepler equation 
$$2 \pi t/T = E - e \sin(E)$$
Where e is orbit eccentricity, t is the time of observation, T is the orbital period. And E is the so-called eccentric anomaly. 
If you know the eccentric anomaly E then the position of the body in the plane of the solar system can be described by 
$$x= a  (\cos E -e) $$
$$y= b \sin E $$
where a, b are the semi-major axis and semi-minor axis respectively.

Your task is to write the function that determines the uncertainty on the position x given the time of observation, ecentricity, period, semi-major axis and their uncertainties.
You can assume that for the eccentric anomaly E will be always between -1000 and 1000.
We will verify that the errors are accurate to within 10%

In [ ]:
def kepler_uncert(t=None, T=None, e=None, a=None, err_t=None, err_T=None, err_e=None, err_a=None):
    # return the value of the uncertainty on the x position of the solar system body
    # YOUR CODE HERE
    
    #this is the size of the distribution
    N = 1000
    
    #generate a normal distribution for variables t,T,e and a. This is given by their mean and error as inputted by the user
    t_time = np.random.normal(t,err_t,size=N)
    T_orbit = np.random.normal(T,err_T,size=N)
    e_ecc = np.random.normal(e,err_e,size=N)
    a = np.random.normal(a,err_a,size=N)
    
    #we need to estimate eccentric anomaly E as it cannot be solved analytically
    #I estimated the mean E to be when sin = 0, therefore E = 2pi * t/T
    mean_E = 2*np.pi*t/T
    #since sin oscillates between -1 and 1, I estimated the deviation in E from its mean is given as +/- e
    #I calculate the error to be the standard deviation over the square root of N
    err_E = e/np.sqrt(N)
    Es = np.random.normal(mean_E, err_E, size=N)
    
    x = a*(np.cos(Es) - e_ecc)
    return x.std()
    

In [ ]:
#the value of the second calculation should be around 0.106

In [ ]:
print(kepler_uncert(t=1, T=2, e=.5, a=3, err_t=1e-5, err_T=1e-4, err_e=1e-3, err_a=0.01))
# This value should be close to 0.015

print(kepler_uncert(t=3, T=4, e=.4, a=6, err_t=1e-4, err_T=1e-3, err_e=1e-2, err_a=0.02))
# This should be close to 0.048


In [ ]:
print(kepler_uncert(t=1, T=2, e=.5, a=3, err_t=1e-5, err_T=1e-4, err_e=1e-3, err_a=0.01))
# This value should be close to 0.015

print(kepler_uncert(t=3, T=4, e=.4, a=6, err_t=1e-4, err_T=1e-3, err_e=1e-2, err_a=0.02))
# This should be close to 0.048


# Task 4 (20 pts)

# Fitting a periodic model to the data. 

In the input data checkpoint2_task4.txt. we have repeated measurements from the instrument. 
The file has 3 columns. One is the time of observation, the second one is the measurement, and the third one is the uncertainty of the measurement. 
We have reasons to believe that the measurement periodically change with time, so we want to fit the dataset provided by a periodic model 
$$M(x) = A \sin(2\pi x/T) + B \cos(2\pi x/T) $$
in order to determine the period T from the data. A,B,T are model parameters

Assuming that we know that the period should be between 0.5 and 5, determine the best period describing the data with the accuracy of better than 0.1.

Your function needs to return the tuple with the best period and bestmodel values

Hint: Use linear regression to fit the periodic models for a grid of periods. Select the period providing the best fit.

In [ ]:
DATA = np.loadtxt('checkpoint2_task4.txt')

plt.errorbar(DATA[0], DATA[1], DATA[2], fmt='.')

#assign variable names to data columns
xdata = DATA[0]
ydata = DATA[1]
sigma = DATA[2]

def findper():
    #periods between 0.5 and 5
    periods = np.linspace(0.5,5,10000)
    
    #define model to be minimized (as given in the question above)
    def Model(x,A,B):
        return A*np.sin(2*np.pi*x/T) + B*np.cos(2*np.pi*x/T)
    
    #initialise empty lists of 1) fitted parameters and 2) chi squared values
    y_fit = []
    chi = []
    
    #fit the periodic models for a grid of periods
    for T in periods:
        #using the curve fitting, fit the model using the given data and initial guesses taken from experimenting with params
        p_opt, p_cov = scipy.optimize.curve_fit(Model, xdata, ydata,p0=[2.83, 2])
        #generate parameter errors to see if they're less than 0.1 (given in question)
        p_err = np.sqrt(np.diag(p_cov))
        #get A and B as the parameters
        p_opt = p_opt.tolist()
        A,B = p_opt
        #also get the period that generates the model parameters
        params = [p_opt, T, p_err]
        #for each iteration add the parameters and period to the y_fit list
        y_fit.append(params)
        #generate chi squared values to see accuracy of the fit
        chi.append(np.sum(np.square(((ydata-Model(xdata,A,B))/sigma))))
        #get the optimum parameters by indexing y_fit where chi squared value is at a minimum
        opt_params = y_fit[np.argmin(chi)]
        
    #get the best period, T
    bestp = opt_params[1]
    #generate the best model by using the optimal parameters
    bestmod = opt_params[0][0]*np.sin(2*np.pi*xdata/bestp) + opt_params[0][1]*np.cos(2*np.pi*xdata/bestp)
    
    #check the chi squared value is reasonable
    print('minimum chi squared value:',min(chi))
    #check the paramater error is reasonable
    print('parameters error:',opt_params[2])
    
    return bestp,bestmod

In [ ]:

# Your code will be tested with this 
plt.errorbar(DATA[0], DATA[1], DATA[2], fmt='.')
bestp, bestmod = findper()
print('BEST PERIOD', bestp)
plt.plot(np.sort(DATA[0]), bestmod[np.argsort(DATA[0])])



# Task 5 (20 pts)

You are given a dataset from the particle physics experiment. 
In that experiment the number of particles are recorded in intervals of Energy.
The dataset contains the array of those numbers and the center Energy of each bin/interval. 
We believe that the average number of particles in each bin could be described by  this model 
$$R(E) = A_1+ A_2 \exp(A_3  E) + A_4  \exp
\left( - \frac{1}{2}\frac{(x-A_5)^2}{A_6^2} \right)$$

where the first two terms are describing the background and the last term is a contribution from a decaying 
particle with the mean energy A5. This contribution creates a peak in the energy distribution. Your task is to measure A5.

In the dataset you can assume that the number of particles in each bin is described by Poisson distribution with the rate described by the equantion above. You can assume that measurement of number of particles in each bin are independent from each other.

Write the likelihood function and estimate the energy of the particle $A_5$ using maximum likelihood approach. 
Overplot your best model on top of the data and make sure it fits well. 

Determine the uncertainty of your measurement. Your function solve_task5() should return two numbers
the estimate of the energy and its uncertainty. (If you don't know how to evaluate uncertainty, you can just return np.nan instead)

In [ ]:
DATA = np.loadtxt('energy.txt', dtype=[('E', np.float64), ('N', int)])
DATA['E']
DATA['N']
plt.plot(DATA['E'], DATA['N'], drawstyle='steps')
plt.xlabel('Energy')
plt.ylabel('Number of particles')

In [ ]:
#assign data to variable names E and N
E = DATA['E']
N = DATA['N']

In [ ]:
#define rate equation as a function of parameters
def rateeqn(E, a1, a2, a3, a4, a5, a6):
    return a1 + a2*np.exp(a3*E)+a4*np.exp(-0.5*((E-a5)**2)/a6**2)

In [ ]:
#the negative log likelyhood function
def get_NLL(a):    
    #unpack unknown parameters a1 through a6
    a1,a2,a3,a4,a5,a6 = a
    LL = []
    for i in range(len(E)):
        #generate the mean, mu, by calculating rate equation for the different energies in dataset
        mu = rateeqn(E[i],a1,a2,a3,a4,a5,a6)
        #"Assume that N in each bin is described by Poisson distribution with the rate described by the equantion above"
        #generate the log pmf of the poisson equation using the calculated mean
        L = scipy.stats.poisson.logpmf(N[i], mu, loc=0)
        #generate list of the log pmfs
        LL.append(L)
    #negative log likely hood is the negative of the sum over all log pmfs
    NLL = -np.sum(LL)
    return NLL

In [ ]:
def solve_task5():
    """
    Your function needs to return the best period and the uncertainty. 
    It also needs to overplot the best model on top of the data
    """
    # YOUR CODE HERE
    #optimize the negative log likelyhood using minimize and generate the estimates for A
    #get results '.x'
    A = scipy.optimize.minimize(get_NLL,x0=[10,10,1,70,70,5]).x
    #get the inverse hessian matrix, '.hess_inv'
    HESS_INV = scipy.optimize.minimize(get_NLL,x0=[10,10,1,70,70,5]).hess_inv
    
    
    #from documentation: '.hess_inv' returned by the fit is the covariance matrix describing the Gaussian approximation of LL
    #he parameter errors are the root of the diag elements of hess_inv
    errs = np.sqrt(np.diag(HESS_INV))
    
    #we want to return a5 and its error, which is A[4] and errs[4] (indexing starts at 0)
    bestval = A[4]
    besterr = errs[4]
    
    return bestval, besterr
    

##### Comments on initial guesses
Initial guesses are chosen by looking at the distribution. `a1` and `a2` account for noise, which is around 20, so I assign them both a value of 10 each. `a3` is the exponential term, since there is not a peak from the noise, I assign it an arbritarily low number of 1. The next term describes a Gaussian distribution. `a4` is given by the height of the peak and `a5` given as the mean (centre of the peak) so I choose them to be 70. `a6` is the width of the peak which is around 5J.


In [ ]:
#get parameter values from function using initial guesses
A = scipy.optimize.minimize(get_NLL,x0=[10,10,1,70,70,5]).x

In [ ]:
#"Overplot your best model on top of the data and make sure it fits well
plt.plot(DATA['E'], DATA['N'], drawstyle='steps')
plt.plot(E,rateeqn(E, A[0], A[1], A[2], A[3], A[4], A[5]),'r')
plt.xlabel('Energy')
plt.ylabel('Number of particles');

In [ ]:
bestval, bessterr = solve_task5()



In [ ]:
bestval, bessterr